# Drug Overdose Modeling

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import patsy 
import math

pd.set_option('display.max_columns', 100)
%matplotlib inline

## Import the feature DataFrame

In [2]:
# Read the DF created from the feature_selection notebook
counties = pd.read_pickle('counties_df.pkl')

In [3]:
# Make the column names standardized for use in patsy formulas
counties.columns = [x.replace(' ', '_').replace('-', '_').replace('%', 'Percent').replace('.', '_').replace('(', '').replace(')', '') for x in list(counties.columns)]
# Patsy also doesn't like column names begining with a number
counties = counties.rename(columns={'20th_Percentile_Income': 'Percentile_Income_20th', '80th_Percentile_Income': 'Percentile_Income_80th'})

## Train a model on the appropriate features

In [4]:
formula = "Drug_Overdose_Mortality_Rate ~ 1 + Percent_Unemployed + Household_Income + Graduation_Rate + Percent_Rural + HIV_Prevalence_Rate + Percent_Frequent_Mental_Distress + Percent_Excessive_Drinking"
y, X = patsy.dmatrices(formula, counties, return_type='matrix') 
model = sm.OLS(y, X)

In [5]:
results = model.fit()

In [6]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     Drug_Overdose_Mortality_Rate   R-squared:                       0.169
Model:                                      OLS   Adj. R-squared:                  0.166
Method:                           Least Squares   F-statistic:                     47.91
Date:                          Mon, 10 Dec 2018   Prob (F-statistic):           3.67e-62
Time:                                  20:04:02   Log-Likelihood:                -6081.7
No. Observations:                          1655   AIC:                         1.218e+04
Df Residuals:                              1647   BIC:                         1.222e+04
Df Model:                                     7                                         
Covariance Type:                      nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                          -19.2633      6.156     -3.129      0.002     -31.337      -7.189
Percent_Unemployed                   0.8768      0.174      5.036      0.000       0.535       1.218
Household_Income                     0.0001    2.9e-05      3.467      0.001    4.37e-05       0.000
Graduation_Rate                      0.0669      0.035      1.924      0.055      -0.001       0.135
Percent_Rural                        0.0172      0.011      1.597      0.110      -0.004       0.038
HIV_Prevalence_Rate                 -0.0027      0.001     -2.109      0.035      -0.005      -0.000
Percent_Frequent_Mental_Distress     2.2124      0.274      8.084      0.000       1.676       2.749
Percent_Excessive_Drinking          -0.2322      0.099     -2.347      0.019      -0.426      -0.038
==============================================================================
Omnibus:                      535.045   Durbin-Watson:                   1.631
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2169.778
Skew:                           1.518   Prob(JB):                         0.00
Kurtosis:                       7.717   Cond. No.                     1.42e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.42e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## See what SciKitLearn thinks are the best features to use

In [7]:
from sklearn.feature_selection import SelectKBest, f_regression
X = counties.copy().drop(columns=['Drug_Overdose_Mortality_Rate'])
X = X.select_dtypes(exclude='object')
y = counties['Drug_Overdose_Mortality_Rate']
kbest = SelectKBest(f_regression, k=20).fit(X, y)

In [8]:
new_features = []
mask = kbest.get_support()

for boolean, feature in zip(mask, list(counties.columns.values)):
    if boolean:
        new_features.append(feature)

In [9]:
new_features

['State',
 'Age_Adjusted_Mortality',
 'Age_Adjusted_Mortality_Black',
 'Child_Mortality_Rate',
 'Child_Mortality_Rate_Black',
 'Child_Mortality_Rate_White',
 'Infant_Mortality_Rate',
 'Percent_Frequent_Physical_Distress',
 'Percent_Uninsured_1',
 'Segregation_index',
 'Percent_Not_Proficient_in_English',
 'State_ranked',
 'Years_of_Potential_Life_Lost_Rate',
 'Years_of_Potential_Life_Lost_Rate_Black',
 'Physically_Unhealthy_Days',
 'Mentally_Unhealthy_Days',
 'Percent_Smokers',
 'Teen_Birth_Rate_White',
 'Percent_Some_College',
 'Percentile_Income_20th']

## Compare our intuitive features choices against the K-best

In [10]:
formula = "Drug_Overdose_Mortality_Rate ~ 1"
for feature in new_features:
    formula += ' + ' + feature
formula

'Drug_Overdose_Mortality_Rate ~ 1 + State + Age_Adjusted_Mortality + Age_Adjusted_Mortality_Black + Child_Mortality_Rate + Child_Mortality_Rate_Black + Child_Mortality_Rate_White + Infant_Mortality_Rate + Percent_Frequent_Physical_Distress + Percent_Uninsured_1 + Segregation_index + Percent_Not_Proficient_in_English + State_ranked + Years_of_Potential_Life_Lost_Rate + Years_of_Potential_Life_Lost_Rate_Black + Physically_Unhealthy_Days + Mentally_Unhealthy_Days + Percent_Smokers + Teen_Birth_Rate_White + Percent_Some_College + Percentile_Income_20th'

In [11]:
y, X = patsy.dmatrices(formula, counties, return_type='matrix') 
model = sm.OLS(y, X)

In [12]:
results = model.fit()

In [13]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     Drug_Overdose_Mortality_Rate   R-squared:                       0.574
Model:                                      OLS   Adj. R-squared:                  0.556
Method:                           Least Squares   F-statistic:                     31.47
Date:                          Mon, 10 Dec 2018   Prob (F-statistic):          9.44e-244
Time:                                  20:04:24   Log-Likelihood:                -5528.4
No. Observations:                          1655   AIC:                         1.119e+04
Df Residuals:                              1586   BIC:                         1.157e+04
Df Model:                                    68                                         
Covariance Type:                      nonrobust                                         
===========================================================================================================
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
Intercept                                 -19.2794      6.375     -3.024      0.003     -31.783      -6.776
State[T.Alaska]                             4.8945      1.924      2.543      0.011       1.120       8.669
State[T.Arizona]                            4.4482      1.278      3.481      0.001       1.942       6.955
State[T.Arkansas]                          -0.6883      0.892     -0.772      0.440      -2.438       1.061
State[T.California]                         4.1262      0.864      4.776      0.000       2.432       5.821
State[T.Colorado]                           5.6241      0.992      5.672      0.000       3.679       7.569
State[T.Connecticut]                        7.7483      1.421      5.453      0.000       4.961      10.535
State[T.Delaware]                           7.0096      2.131      3.290      0.001       2.830      11.189
State[T.District of Columbia]               3.2218      3.595      0.896      0.370      -3.830      10.273
State[T.Florida]                            4.4709      0.868      5.153      0.000       2.769       6.173
State[T.Georgia]                            1.8464      0.855      2.158      0.031       0.168       3.524
State[T.Hawaii]                             3.1186      1.912      1.631      0.103      -0.632       6.869
State[T.Idaho]                              3.3369      1.305      2.557      0.011       0.778       5.896
State[T.Illinois]                           4.0441      0.860      4.703      0.000       2.357       5.731
State[T.Indiana]                            4.1535      0.863      4.814      0.000       2.461       5.846
State[T.Iowa]                               2.6893      1.124      2.393      0.017       0.485       4.893
State[T.Kansas]                             2.3015      1.181      1.948      0.052      -0.016       4.619
State[T.Kentucky]                           8.1844      0.761     10.759      0.000       6.692       9.676
State[T.Louisiana]                          1.9388      0.841      2.306      0.021       0.290       3.588
State[T.Maine]                              5.6268      1.150      4.894      0.000       3.371       7.882
State[T.Maryland]                           6.5947      0.985      6.697      0.000       4.663       8.526
State[T.Massachusetts]                      7.6709      1.205      6.366      0.000       5.307      10.034
State[T.Michigan]                           4.8377      0.784      6.167      0.000       3.299       6.376
State[T.Minnesota]                          3.4729      1.041      3.337      0.001       1.432       5.514
State[T.Mississippi]                       -1.3768      0